In [41]:
# Retrieve packages
from splinter import Browser
from bs4 import BeautifulSoup
import pandas as pd
import time
import re

from sqlalchemy import create_engine, MetaData
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, String, Integer, Float
from sqlalchemy.orm import sessionmaker


1. Create DB

In [66]:
# Set up Player_info table

Base = declarative_base()

class Player_info_table(Base):
    __tablename__ = "Player_info_table"
    name = Column(String(30), primary_key=True)
    age = Column(String(15))
    current_team = Column(String(50))
    height = Column(String(15))
    weight = Column(String(15))
    nba_status = Column(String(50))
    draft_entry = Column(String(50))


In [43]:
# Set up Team table

Base = declarative_base()

class Team_table(Base):
    __tablename__ = "Team_table"
    team = Column(String(50), primary_key=True)
    gp = Column(Float)
    mpg = Column(Float)
    ppg = Column(Float)
    fmg = Column(Float)
    fga = Column(Float)
    fg_prc = Column(Float)
    three_pm = Column(Float)
    three_pa = Column(Float)
    three_p_prc = Column(Float)
    ftm = Column(Float)
    fta = Column(Float)
    ft_prc = Column(Float)
    orb = Column(Float)
    drb = Column(Float)
    rpg = Column(Float)
    apg = Column(Float)
    spg = Column(Float)
    bpg = Column(Float)
    tov = Column(Float)
    pf = Column(Float)



In [67]:
# Create engine to DB

engine = create_engine("sqlite:///NBA_stats.sqlite")
Base.metadata.create_all(engine)

Session = sessionmaker(bind=engine)
session = Session()


2. Scrape website to retrieve Player Data

In [54]:
# Define page
browser = Browser('chrome')
url = "https://basketball.realgm.com/nba/stats"
browser.visit(url)
time.sleep(5)
html = browser.html
soup = BeautifulSoup(html, 'html.parser')
browser.quit()


In [55]:
# Retrieve Players
response = soup.find_all(class_="nowrap tablesaw-cell-persist")

players = []
url_player = []

for x in range(len(response)):
    players.append(response[x].text)

    text = response[x]
    href_pattern = r'<a href="([^"]+)">([^<]+)</a>'
    matches = re.findall(href_pattern, str(text))
    href_text = [match[0] for match in matches]
    href_text = href_text[0]
    href_text = "https://basketball.realgm.com/" + href_text
    url_player.append(href_text)



In [56]:
# Find Current Team Function

def find_current_team(soup):
    response = soup.find_all(class_="half-column-left")
     
    current_team_pattern = r'<strong>Current Team:</strong>\s*<a href="[^"]+">([^<]+)</a>'
    match = re.search(current_team_pattern, str(response[0]))
    
    if match:
        current_team = match.group(1)
    else:
        current_team = None
    
    return current_team



In [57]:
# Find age Function

def find_age(soup):
    response = soup.find_all(class_="half-column-left")
        
    age_pattern = r'<strong>Born:</strong>\s*<a href="[^"]+">[^<]+</a>\s*\(([^)]+)\)'
    match = re.search(age_pattern, str(response[0]))
    
    if match:
        age = match.group(1)
    else:
        age = None
    
    return age


In [58]:
# Find height Function

def find_height(soup):
    response = soup.find_all(class_="half-column-left")
        
    height_pattern = r'<strong>Height:</strong>\s*([^<]+)'
    match = re.search(height_pattern, str(response[0]))
    
    if match:
        height = match.group(1).strip()
    else:
        height = None
    
    return height

In [59]:
# Find weight Function

def find_weight(soup):
    response = soup.find_all(class_="half-column-left")
        
    weight_pattern = r'<strong>Weight:</strong>\s*([^<]+)'
    match = re.search(weight_pattern, str(response[0]))
    
    if match:
        weight = match.group(1).strip()
    else:
        weight = None
    
    return weight

In [60]:
# Find current NBA status Function

def find_nba_status(soup):
    response = soup.find_all(class_="half-column-right")
        
    nba_status_pattern = r'<strong>Current NBA Status:</strong>\s*([^<]+)'
    match = re.search(nba_status_pattern, str(response[0]))
    
    if match:
        nba_status = match.group(1).strip()
    else:
        nba_status = None
    
    return nba_status



In [61]:
# Find draft entry Function

def find_draft_entry(soup):
    response = soup.find_all(class_="half-column-right")
        
    draft_entry_pattern = r'<strong>Draft Entry:</strong>\s*<a[^>]+>([^<]+)</a>'
    match = re.search(draft_entry_pattern, str(response[0]))
    
    if match:
        draft_entry = match.group(1).strip()
    else:
        draft_entry = None
    
    return draft_entry

In [74]:
# Retrieve score function

def find_scores(soup):
    response = soup.find_all(class_="per_game")
    
    year_overview = []
    gp_overview = []
    gs_overview = []
    min_overview = []
    pts_overview = []
    fgm_overview = []
    fga_overview = []
    fg_prc_overview = []
    three_pm_overview = []
    three_pa_overview = []
    three_p_prc_overview = []
    ftm_overview = []
    fta_overview = []
    ft_prc_overview = []
    off_overview = []
    deff_overview = []
    trb_overview = []
    ast_overview = []
    stl_overview = []
    blk_overview = []
    tov_overview = []
    pf_overview = []

    for x in range(len(response)-1):
        try:
            overview_scores = response[x+1].find_all('td')
            year = overview_scores[0].text
            year = year[:7]
            gp = overview_scores[2]
            gs = overview_scores[3]
            min = overview_scores[4]
            pts = overview_scores[5]
            fgm = overview_scores[6]
            fga = overview_scores[7]
            fg_prc = overview_scores[8]
            three_pm = overview_scores[9]
            three_pa = overview_scores[10]
            three_p_prc = overview_scores[11]
            ftm = overview_scores[12]
            fta = overview_scores[13]
            ft_prc = overview_scores[14]
            off = overview_scores[15]
            deff = overview_scores[16]
            trb = overview_scores[17]
            ast = overview_scores[18]
            stl = overview_scores[19]
            blk = overview_scores[20]
            tov = overview_scores[21]
            pf = overview_scores[22]

        except:
            pass

        if year[:2] == "20":
            year_overview.append(year)
            gp_overview.append(gp)
            gs_overview.append(gs)
            min_overview.append(min)
            pts_overview.append(pts)
            fgm_overview.append(fgm)
            fga_overview.append(fga)
            fg_prc_overview.append(fg_prc)
            three_pm_overview.append(three_pm)
            three_pa_overview.append(three_pa)
            three_p_prc_overview.append(three_p_prc)
            ftm_overview.append(ftm)
            fta_overview.append(fta)
            ft_prc_overview.append(ft_prc)
            off_overview.append(off)
            deff_overview.append(deff)
            trb_overview.append(trb)
            ast_overview.append(ast)
            stl_overview.append(stl)
            blk_overview.append(blk)
            tov_overview.append(tov)
            pf_overview.append(pf)
            
        else:
            pass
        
    find_scores_df = pd.DataFrame({"Year":year_overview,
                                   "GP":gp_overview,
                                   "GS": gs_overview,
                                   "MIN": min_overview,
                                   "PTS": pts_overview,
                                   "FGM": fgm_overview,
                                   "FGA": fga_overview,
                                   "FG%": fg_prc_overview,
                                   "3PM": three_pm_overview,
                                   "3PA": three_pa_overview,
                                   "3P%": three_p_prc_overview,
                                   "FTM": ftm_overview,
                                   "FTA": fta_overview,
                                   "FT%": ft_prc_overview,
                                   "OFF": off_overview,
                                   "DEF": deff_overview,
                                   "TRB": trb_overview,
                                   "AST": ast_overview,
                                   "STL": stl_overview,
                                   "BLK": blk_overview,
                                   "TOV": tov_overview,
                                   "PF": pf_overview})
    
    
    duplicate_mask = find_scores_df['Year'].duplicated()
    keep_mask = ~duplicate_mask
    find_scores_df = find_scores_df[keep_mask]

    find_scores_df['Year'] = find_scores_df['Year'].astype(str)

    rows_with_value = find_scores_df.index[find_scores_df['Year'] == '2022-23'].tolist()
    find_scores_df = find_scores_df.head(rows_with_value[0]+1)


    return find_scores_df



In [78]:
# Browse to player page and retrieve data

player_overview = []
current_team_overview = []
age_overview = []
height_overview = []
weight_overview =[]
nba_status_overview =[]
draft_entry_overview = []

player_scores_df = pd.DataFrame({"Player":[],
                                 "Year":[],
                                 "GP":[],
                                 "GS": [],
                                 "MIN": [],
                                 "PTS": [],
                                 "FGM": [],
                                 "FGA": [],
                                 "FG%": [],
                                 "3PM": [],
                                 "3PA": [],
                                 "3P%": [],
                                 "FTM": [],
                                 "FTA": [],
                                 "FT%": [],
                                 "OFF": [],
                                 "DEF": [],
                                 "TRB": [],
                                 "AST": [],
                                 "STL": [],
                                 "BLK": [],
                                 "TOV": [],
                                 "PF": []})


for x in range(len(players)):
    browser = Browser('chrome')
    url = url_player[x]
    browser.visit(url)
    time.sleep(5)
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')
    browser.quit()

    player_overview.append(players[x])
    current_team_overview.append(find_current_team(soup))
    age_overview.append(find_age(soup))
    height_overview.append(find_height(soup))
    weight_overview.append(find_weight(soup))
    nba_status_overview.append(find_nba_status(soup))
    draft_entry_overview.append(find_draft_entry(soup))

    player_temp_df = find_scores(soup)

    player_name_overview = []

    for x in range(len(player_temp_df)):
        player_name_overview.append(players[x])

    player_temp_df["Player"] = player_name_overview
    
    player_scores_df = pd.concat([player_scores_df, player_temp_df], ignore_index=True)


# Create Player Info Dataframe

player_info_df = pd.DataFrame({"Player": player_overview,
                               "Current team": current_team_overview,
                               "Age": age_overview,
                               "Height": height_overview,
                               "Weight": weight_overview,
                               "NBA status": nba_status_overview,
                               "Draft entry": draft_entry_overview})



3. Scrape website to retrieve Team Data

In [4]:
# Define page
browser = Browser('chrome')
url = "https://basketball.realgm.com/nba/team-stats"
browser.visit(url)
time.sleep(5)
html = browser.html
soup = BeautifulSoup(html, 'html.parser')
browser.quit()

In [6]:
# Retrieve team & stats

team_response = soup.find_all('tr')

team_overview = []
gp_overview =[]
mpg_overview = []
ppg_overview = []
fgm_overview =[]
fga_overview =[]
fg_prc_overview = []
three_pm_overview =[]
three_pa_overview =[]
three_p_prc = []
ftm_overview =[]
fta_overview =[]
ft_prc_overview =[]
orb_overview = []
drb_overview = []
rpg_overview =[]
apg_overview = []
spg_overview = []
bpg_overview =[]
tov_overview = []
pf_overview =[]


for x in range(6,36):
    response = team_response[x].find_all('td')
    team_overview.append(response[1].text)
    gp_overview.append(response[2].text)
    mpg_overview.append(response[3].text)
    ppg_overview.append(response[4].text)
    fgm_overview.append(response[5].text)
    fga_overview.append(response[6].text)
    fg_prc_overview.append(response[7].text)
    three_pm_overview.append(response[8].text)
    three_pa_overview.append(response[9].text)
    three_p_prc.append(response[10].text)
    ftm_overview.append(response[11].text)
    fta_overview.append(response[12].text)
    ft_prc_overview.append(response[13].text)
    orb_overview.append(response[14].text)
    drb_overview.append(response[15].text)
    rpg_overview.append(response[16].text)
    apg_overview.append(response[17].text)
    spg_overview.append(response[18].text)
    bpg_overview.append(response[19].text)
    tov_overview.append(response[20].text)
    pf_overview.append(response[21].text)

In [7]:
# Create dataframe of results

team_scores_df = pd.DataFrame({"Team": team_overview,
                               "GP": gp_overview,
                               "MPG": mpg_overview,
                               "PPG": ppg_overview,
                               "FMG": fgm_overview,
                               "FGA": fga_overview,
                               "FG%": fg_prc_overview,
                               "3PM": three_pm_overview,
                               "3PA": three_pa_overview,
                               "3P%": three_p_prc,
                               "FTM": ftm_overview,
                               "FTA": fta_overview,
                               "FT%": ft_prc_overview,
                               "ORB": orb_overview,
                               "DRB": drb_overview,
                               "RPG": rpg_overview,
                               "APG": apg_overview,
                               "SPG": spg_overview,
                               "BPG": bpg_overview,
                               "TOV": tov_overview,
                               "PF": pf_overview})
team_scores_df.head()

,Team,GP,MPG,PPG,FMG,FGA,FG%,3PM,3PA,3P%,...,FTA,FT%,ORB,DRB,RPG,APG,SPG,BPG,TOV,PF
0,Sacramento,82,48.4,120.7,43.6,88.2,.494,13.8,37.3,.369,...,25.1,.790,9.5,32.9,42.5,27.3,7.0,3.4,13.5,19.7
1,Golden State,82,48.4,118.9,43.1,90.2,.479,16.6,43.2,.385,...,20.2,.794,10.5,34.1,44.6,29.8,7.2,3.9,16.3,21.4
2,Atlanta,82,48.4,118.4,44.6,92.4,.483,10.8,30.5,.352,...,22.6,.818,11.2,33.2,44.4,25.0,7.1,4.9,12.9,18.8
3,Boston,82,48.7,117.9,42.2,88.8,.475,16.0,42.6,.377,...,21.6,.812,9.7,35.6,45.3,26.7,6.4,5.2,13.4,18.8
4,Oklahoma City,82,48.4,117.5,43.1,92.6,.465,12.1,34.1,.356,...,23.7,.809,11.4,32.3,43.6,24.4,8.2,4.2,13.0,21.0
5,L.A. Lakers,82,48.5,117.2,42.9,89.0,.482,10.8,31.2,.346,...,26.6,.775,10.0,35.7,45.7,25.3,6.4,4.6,14.1,17.9
6,Utah,82,48.3,117.1,42.5,89.8,.473,13.3,37.8,.353,...,23.8,.786,11.8,34.1,45.9,26.0,6.1,5.2,15.3,20.5
7,Milwaukee,82,48.4,116.9,42.7,90.4,.473,14.8,40.3,.368,...,22.4,.743,11.1,37.5,48.6,25.8,6.4,4.9,14.6,18.1
8,Memphis,82,48.2,116.9,43.7,92.1,.475,12.0,34.2,.351,...,23.8,.733,12.0,34.6,46.6,26.0,8.3,5.8,13.6,20.0
9,Indiana,82,48.2,116.3,42.0,89.6,.469,13.6,37.0,.367,...,23.7,.790,10.1,31.4,41.5,27.0,7.7,5.8,14.9,21.2


4. Save data in DB

In [71]:
# Clear existing data from DB

session.execute(Team_table.__table__.delete())
session.execute(Player_info_table.__table__.delete())
session.commit()


In [70]:
# Add Player Info Data

for index, row in player_info_df.iterrows():
    team_update = Player_info_table(
        name=row[0],
        current_team=row[1],
        age=row[2],
        height=row[3],
        weight=row[4],
        nba_status=row[5],
        draft_entry=row[6]
    )
    session.add(team_update)

session.commit()


In [47]:
# Add Team Data

for index, row in team_scores_df.iterrows():
    team_update = Team_table(
        team=row[0],
        gp=row[1],
        mpg=row[2],
        ppg=row[3],
        fmg=row[4],
        fga=row[5],
        fg_prc=row[6],
        three_pm=row[7],
        three_pa=row[8],
        three_p_prc=row[9],
        ftm=row[10],
        fta=row[11],
        ft_prc=row[12],
        orb=row[13],
        drb=row[14],
        rpg=row[15],
        apg=row[16],
        spg=row[17],
        bpg=row[18],
        tov=row[19],
        pf=row[20]
    )
    session.add(team_update)

session.commit()


In [53]:
# Test Team Upload

session.query(Team_table.team, Team_table.gp, Team_table.mpg).all()




[('Sacramento', 82.0, 48.4),
 ('Golden State', 82.0, 48.4),
 ('Atlanta', 82.0, 48.4),
 ('Boston', 82.0, 48.7),
 ('Oklahoma City', 82.0, 48.4),
 ('L.A. Lakers', 82.0, 48.5),
 ('Utah', 82.0, 48.3),
 ('Milwaukee', 82.0, 48.4),
 ('Memphis', 82.0, 48.2),
 ('Indiana', 82.0, 48.2),
 ('New York', 82.0, 48.7),
 ('Denver', 82.0, 48.2),
 ('Minnesota', 82.0, 48.4),
 ('Philadelphia', 82.0, 48.5),
 ('New Orleans', 82.0, 48.4),
 ('Dallas', 82.0, 48.6),
 ('Phoenix', 82.0, 48.2),
 ('L.A. Clippers', 82.0, 48.4),
 ('Portland', 82.0, 48.1),
 ('Brooklyn', 82.0, 48.1),
 ('Washington', 82.0, 48.2),
 ('Chicago', 82.0, 48.5),
 ('San Antonio', 82.0, 48.4),
 ('Toronto', 82.0, 48.3),
 ('Cleveland', 82.0, 48.5),
 ('Orlando', 82.0, 48.2),
 ('Charlotte', 82.0, 48.4),
 ('Houston', 82.0, 48.2),
 ('Detroit', 82.0, 48.3),
 ('Miami', 82.0, 48.3)]

In [73]:
# Test Player Info Upload

session.query(Player_info_table.name, Player_info_table.current_team, Player_info_table.age).all()


[('Joel Embiid', 'Philadelphia Sixers', '29 years old'),
 ('Luka Doncic', 'Dallas Mavericks', '24 years old'),
 ('Damian Lillard', 'Portland Trail Blazers', '32 years old'),
 ('Shai Gilgeous-Alexander', 'Oklahoma City Thunder', '24 years old'),
 ('Giannis Antetokounmpo', 'Milwaukee Bucks', '28 years old'),
 ('Jayson Tatum', 'Boston Celtics', '25 years old'),
 ('Stephen Curry', 'Golden State Warriors', '35 years old'),
 ('Kevin Durant', 'Phoenix Suns', '34 years old'),
 ('LeBron James', 'Los Angeles Lakers', '38 years old'),
 ('Donovan Mitchell', 'Cleveland Cavaliers', '26 years old'),
 ('Devin Booker', 'Phoenix Suns', '26 years old'),
 ('Kyrie Irving', 'Dallas Mavericks', '31 years old'),
 ('Jaylen Brown', 'Boston Celtics', '26 years old'),
 ('Trae Young', 'Atlanta Hawks', '24 years old'),
 ('Ja Morant', 'Memphis Grizzlies', '23 years old'),
 ('Anthony Davis', 'Los Angeles Lakers', '30 years old'),
 ('Lauri Markkanen', 'Utah Jazz', '26 years old'),
 ('Julius Randle', 'New York Knicks',

In [ ]:
# Close session

session.close()